In [1]:
import re,os
from collections import Counter
import itertools
import numpy as np

#utility functions block

class Symb(dict):
    def valmult(self,m,d1):
        return {k:(m*v) for k,v in d1.items()}
    
    def dictmerge(self,d1,d2):
        def getval(k):
            if k in d1 and k in d2: val= d1[k]+d2[k]
            elif k in d1: val= d1[k]
            elif k in d2: val= d2[k]
            else: val = None
            return val

        return{k:getval(k) for k in d1|d2 if getval(k) != 0}
    
    def dictdiff(self,d1,d2):
        def getval(k):
            if k in d1 and k in d2: val= d1[k]-d2[k]
            elif k in d1: val= d1[k]
            elif k in d2: val= -d2[k]
            else: val = None
            return val
    
        return{k:getval(k) for k in d1|d2 if getval(k) != 0}
    
    def __add__(self, othersymb):
        if isinstance(othersymb,Symb): os=othersymb
        else: os=Symb(othersymb)
        s=self.dictmerge(self,os)
        #print(Symb(s))
        #print(s,os.mydict,self.mydict)
        return Symb(s)
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, othersymb):
        return Symb(self.dictdiff(self,othersymb))

    def __rsub__(self, othersymb):
        return Symb(self.dictdiff(othersymb,self))

    def __mul__(self,const):
        return Symb(self.valmult(const,self))
    
    def __rmul__(self,const):
        return Symb(self.valmult(const,self))
    
    def __getitem__(self,key):
        if key in self: return super().__getitem__(key)
        else: return 0

In [69]:
steinmanns={'a':'d', 'b':'e', 'c':'f', 'd':'aef', 'e':'bdf', 'f':'cde'}

def is_triv_zero(word):
    for i,letter in enumerate(word):
        if i == len(word)-1: continue
        else:
            if word[i+1] in steinmanns[letter]: return True
    return False

def is_ok_phi2(word):
    if word[0] in 'def': return False
    elif word[-1] in 'abc': return False
    elif is_triv_zero(word): return False
    else: return True

def is_ok_phi3(word):
    if word[0] in 'def': return False
    elif word[-1] in 'def': return False
    elif is_triv_zero(word): return False
    else: return True

def is_ok(word):
    return True

def count_ones(binlist):
    i=0
    for elem in binlist:
        if elem == 1: i += 1
    return i

def count_zeros(binlist):
    i=0
    for elem in binlist:
        if elem == 0: i += 1
    return i

In [70]:
def readESymb(loop, name, prefix):
    assert os.path.isfile(name)
    res = ''
    prefix = prefix

    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(loop) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res

def convert(loop, filename,prefix):
    dev = re.split(":=|SB\(|\)", re.sub('[,*]', '', readESymb(loop, filename, prefix)))[1:-1]
    keys = dev[1::2]
    values = [int(re.sub('[+-]$', t[0] + '1', t)) if t else 1 for t in dev[0::2]]
    out_dict = {}
    for k, v in zip(keys, values):
        out_dict[k] = v
    return out_dict

symbs={}
for L in [1,2,3,4,5,6]:
    if L==6:
        symbs[L]=convert(L,'../data/EZ6_symb_new_norm','Esymb')
    else: symbs[L]=convert(L,'../data/EZ_symb_new_norm','Esymb')
    
    
symbs3={}
for L in [1,2,3,4,5]:
    if L==6:
        symbs3[L]=convert(L,'../data/EE33_symb_new_norm','EE33_6_symb_new_norm')
    else: symbs3[L]=convert(L,'../data/EE33_symb_new_norm','EE33_symb_new_norm')

In [71]:
def arity(word):
    return word.count('a')+word.count('b')+word.count('c')

def a_arity(word):
    return word.count('a')

In [72]:
def kbits(totsum, str_len):
    for bits in itertools.combinations(range(str_len), totsum):
        s = [0] * str_len
        for bit in bits:
            s[bit] = 1
        yield s

def StuffleIterator(_l1,_l2,r,opt=None):

    m = len(_l1)
    n = len(_l2)

    wc1, wc2 = _l1, _l2

    blank = [None] * (m + n - r)
    for iv in kbits(m, m + n - r):
        if opt == "left":
            if iv[0] != 1:
                continue
        if opt == "right":
            if iv[-1] != 0:
                continue
        if opt == "leftright":
            if iv[0] != 1:
                continue
            if iv[-1] != 0:
                continue       
        w = blank[:]
        filled_places = []
        unfilled_places = []
        # Fill in w1 into the iv slots
        i = 0
        for j in range(len(iv)):
            if iv[j] == 1:
                w[j] = wc1[i]
                i += 1
                filled_places.append(j)
            else:
                unfilled_places.append(j)
        # Choose r of these filled places
        for subset in itertools.combinations(filled_places, r):
            places_to_fill = sorted(unfilled_places + list(subset))

            # Fill in w2 into the places
            i = 0
            res = w[:]
            for j in places_to_fill:
                if res[j] is not None:
                    res[j] = "("+res[j]+'+'+wc2[i]+")"
                else:
                    res[j] = wc2[i]
                i += 1
            yield [i for i in res if i]

            
def ShuffleProduct(l1,l2,cond=None):
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str)
    stufslist=[''.join(shuf) for shuf in StuffleIterator(l1,l2,r=0) if func(''.join(shuf))]
    return Symb(dict(Counter(stufslist)))
            
def ShuffleProduct_overlapping_r(l1,l2,r,cond=None):
    """
    The overlapping shuffle product of the two words ``w1`` and ``w2``
    with precisely ``r`` overlaps.
    """
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str) and isinstance(r,int)
    _l1=list(l1)
    _l2=list(l2)
                
    return Symb(dict(Counter([''.join(shuf) for shuf in StuffleIterator(_l1,_l2,r) if func(''.join(shuf))])))

def StuffleProduct(l1,l2,cond=None):
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str)
    stufslist=[]
    for r in range(min(len(l1), len(l2)) + 1):
        stufslist+=[''.join(shuf) for shuf in StuffleIterator(l1,l2,r) if func(''.join(shuf))]
    return Symb(dict(Counter(stufslist)))

def LHalfShuffleProduct(l1,l2,cond=None):
    #all shuffles that begin like l1
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str)
    stufslist=[''.join(shuf) for shuf in StuffleIterator(l1,l2,r=0, opt="left") if func(''.join(shuf))]
    return Symb(dict(Counter(stufslist)))

def RHalfShuffleProduct(l1,l2,cond=None):
    #all shuffles that end like l2
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str)
    stufslist=[''.join(shuf) for shuf in StuffleIterator(l1,l2,r=0, opt="right") if func(''.join(shuf))]
    return Symb(dict(Counter(stufslist)))

def LRHalfShuffleProduct(l1,l2,cond=None):
    if cond == "ok_phi2": func= is_ok_phi2   
    elif cond == "ok_phi3": func = is_ok_phi3
    else: func = is_ok
    assert isinstance(l1, str) and isinstance(l2, str)
    stufslist=[''.join(shuf) for shuf in StuffleIterator(l1,l2,r=0, opt="leftright") if func(''.join(shuf))]
    return Symb(dict(Counter(stufslist)))

def LeftPreLieShuffleProduct(l1,l2,cond=None):
    return LHalfShuffleProduct(l1,l2,cond)-RHalfShuffleProduct(l2,l1,cond)

def RightPreLieShuffleProduct(l1,l2,cond=None):
    return RHalfShuffleProduct(l1,l2,cond)-LHalfShuffleProduct(l2,l1,cond)

def shufsymbs(symb1,symb2,cond=None):
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*ShuffleProduct(k1,k2,cond=cond)
            shufs = shufs + shuf
    return shufs

def Rshufsymbs(symb1,symb2,cond=None):
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*RHalfShuffleProduct(k1,k2,cond=cond)     
            shufs = shufs + shuf
    return shufs

def Lshufsymbs(symb1,symb2,cond=None):
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*LHalfShuffleProduct(k1,k2,cond=cond)       
            shufs = shufs + shuf
    return shufs

def LRshufsymbs(symb1,symb2,cond=None):
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*LRHalfShuffleProduct(k1,k2,cond=cond)      
            shufs = shufs + shuf
    return shufs

def LeftPreLieShufSymbs(symb1,symb2,cond=None):
    #check
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*(LHalfShuffleProduct(k1,k2,cond=cond)-RHalfShuffleProduct(k2,k1,cond=cond))
            shufs = shufs + shuf
    return shufs

def RightPreLieShufSymbs(symb1,symb2,cond=None):
    #check
    shufs={}
    for k1,v1 in symb1.items():
        for k2,v2 in symb2.items():
            shuf=v1*v2*(RHalfShuffleProduct(k1,k2,cond=cond)-LHalfShuffleProduct(k2,k1,cond=cond))
            shufs = shufs + shuf
    return shufs

In [73]:
def UnshuffleProduct(shuffled_dict,in_dict,opt=None,cond=None,guess=False):
    #Need to be careful since L and R unshufs aren't commutative.
    #Defining a convention here: if A (LeftShuf) B = C,
    #Then B = C (LeftUnshufFirst) A, and  A = C (LeftUnshufSecond) B
    
    unshuf_counters={}
    unshuf_dicts={}
    all_possible_keys={}
    #get a word in the shuffle output.
    for long_word in shuffled_dict:
        unshuf_counter={}
        unshuf_dict={}
        for k in in_dict:
            m = len(long_word)
            n = len(k)

            wc1, wc2 = long_word, k

            blank = [None] * (m - n)
            #Check that we have at least one valid way to get it with shuffles.
            #Gets: unshuffle keys and multiplicity.  
            for iv in kbits(m-n, m):
                if opt == "left_first":
                    if iv[0] != 0:
                        continue
                if opt == "right_first":
                    if iv[-1] != 1:
                        continue
                if opt == "left_second":
                    if iv[0] != 1:
                        continue
                if opt == "right_second":
                    if iv[-1] != 0:
                        continue
                if opt == "leftright_first":
                    if iv[0] != 0:
                        continue
                    if iv[-1] != 1:
                        continue 
                if opt == "leftright_second":
                    if iv[0] != 1:
                        continue
                    if iv[-1] != 0:
                        continue 
                        
                if count_zeros(iv) != len(k): continue
                # Fill in w1 into the iv slots
                i = 0
                is_bad=False
                unshuf_seq=[]
                for j in range(len(iv)):
                    if (iv[j] == 0):
                        #if we hit a 0 and the sequences line up, we're good.
                        #otherwise, throw away this kbits and get the next one
                        if wc1[j] == wc2[i]: 
                            i += 1
                        else:
                            is_bad= True
                    if (iv[j] == 1):
                        unshuf_seq += wc1[j]
                if not is_bad:
                    goodkey=''.join(unshuf_seq)
                    if not goodkey in unshuf_counter:
                        unshuf_dict[goodkey]=k
                        unshuf_counter[goodkey] = 1
                        if not goodkey in all_possible_keys:
                            all_possible_keys[goodkey]=1
                    else:
                        unshuf_counter[goodkey] += 1
            
            #for each word, store the possible unshuffles and the multiplicity
            unshuf_counters[long_word]=unshuf_counter
            unshuf_dicts[long_word]=unshuf_dict
    
    def get_in_coef(long_word,outkey):
        #get the coef of the input word corresponding to a given shuffle candidate,
        #times the shuffle multiplicity 
        if outkey in unshuf_dicts[long_word]:
            in_coef=unshuf_counters[long_word][outkey]*in_dict[unshuf_dicts[long_word][outkey]]
        else:
            #print(unshuf_counters[long_word])
            in_coef=0
        return in_coef
            
    #We know that shuffle_multiplicity_1*(in_coeff_1,shuf_coef_1)+ ... +
    #shuffle_multiplicity_n*(in_coeff_n,shuf_coef_n) = out_coeff.
    #so we need to solve a system. Construct the input matrix: [N_shuffled_words*N_unshuffle_candidates]
    M,y=[],[]
    for i,long_word in enumerate(shuffled_dict):
        y.append(shuffled_dict[long_word])
        M.append([get_in_coef(long_word,key) for key in all_possible_keys.keys()])

    npM=np.array(M)
    npy=np.array(y)
    
    #print(npM,npy)
    outs=np.round(np.linalg.lstsq(npM,npy,rcond=None)[0]).astype(int)
    unshuf_dict={}
    for i,key in enumerate(all_possible_keys.keys()):
        if outs[i] == 0: continue
        else: unshuf_dict[key] = outs[i]
    
    #Check:
    if opt == "left_first":
        mysymb= Lshufsymbs(Symb(in_dict),Symb(unshuf_dict),cond=cond)
    elif opt == "right_first":
        mysymb= Rshufsymbs(Symb(in_dict),Symb(unshuf_dict),cond=cond)
    elif opt == "left_second":
        mysymb= Lshufsymbs(Symb(unshuf_dict),Symb(in_dict),cond=cond)
    elif opt == "right_second":
        mysymb= Rshufsymbs(Symb(unshuf_dict),Symb(in_dict),cond=cond)
    elif opt == "leftright_first":
        mysymb= LRshufsymbs(Symb(in_dict),Symb(unshuf_dict),cond=cond)
    elif opt == "leftright_second":
        mysymb= LRshufsymbs(Symb(unshuf_dict),Symb(in_dict),cond=cond)        
    else:
        mysymb= shufsymbs(Symb(unshuf_dict),Symb(in_dict),cond=cond)
        
    if mysymb != Symb(shuffled_dict):
        print("No solution!")
        if not guess: return None
    return Symb(unshuf_dict)

In [75]:
#print(Lshufsymbs({'ab':1},{'dbb': 1, 'cdd': 1},cond='ok_phi2'))

abc_symb={'aa':1,'bb':1,'cc':1}
def_symb={'dd':1,'ee':1,'ff':1}

#guess=UnshuffleProduct(symbs[2],symbs[1],cond='ok_phi2',opt='right_first',guess=True)

print(guess)
print(shufsymbs(guess,symbs[1],cond='ok_phi2'))
print(symbs[2])
#LShuf: begins with first of leftmost key 
#Rshuf: ends with last of rightmost key

{'dd': -2, 'ee': -2, 'ff': -2}
{'bddd': 12, 'cddd': 12, 'aeee': 12, 'ceee': 12, 'afff': 12, 'bfff': 12}
{'aaae': 16, 'aaaf': 16, 'bbbd': 16, 'bbbf': 16, 'cccd': 16, 'ccce': 16, 'bddd': 8, 'cddd': 8, 'ceee': 8, 'aeee': 8, 'afff': 8, 'bfff': 8}


In [36]:



UnshuffleProduct(Lshufsymbs({'dbb': 1, 'cdd': 1},{'ab':1}),{'ab':1},opt="left")
#print(shufsymbs({'dbb': 1, 'cdd': 1},{'ab':1}))
#UnshuffleProduct({'bdab': 1, 'abdb': 2, 'badb': 1, 'abbd': 2, 'babd': 1, 'dbab': 1, 'adbb': 2, 'dabb': 2, 'cdab': 2, 'acdb': 2, 'cadb': 2, 'abcd': 2, 'acbd': 2, 'cabd': 2}, {"ab":1})

No solution! Best guess is:


{'dbb': 1, 'cdd': 1}

In [151]:
UnshuffleProduct(symbs[3],symbs[2])

No solution! Best guess is:


{'bb': -2, 'aa': -2, 'ff': -1, 'dd': -1, 'ee': -1, 'cc': -2}

In [128]:
def UnshuffleSymbs(long_symb,short_symb,opt=None):

    #get a word in the shuffle output.    
    for long_word in multi_word:
        m = len(long_word)
        n = len(short_word)

        wc1, wc2 = _l1, _l2

        blank = [None] * (m - n)
        #Check that we have at least one valid way to get it with shuffles
        for iv in kbits(m-n, m):
            #for each sequence: if 
            filled_places = []
            unfilled_places = []
            # Fill in w1 into the iv slots
            i = 0
            is_bad=False
            unshuf_seq=[]
            for j in range(len(iv)):
                if (iv[j] == 1):
                    #if we hit a 1 and the sequences line up, we're good.
                    #otherwise, throw away this kbits and get the next one
                    if wc1[j] == wc2[i]: 
                        i += 1
                    else:
                        is_bad= True
                if (iv[j] == 0):
                    unshuf_seq += wc1[j]
            if not is_bad:
                #if it's a valid unshuffle, store the word.
                #If it is stored already, increment its counter
                
                
                
            
                
                
        # Choose r of these filled places
        for subset in itertools.combinations(filled_places, r):
            places_to_fill = sorted(unfilled_places + list(subset))

            # Fill in w2 into the places
            i = 0
            res = w[:]
            for j in places_to_fill:
                if res[j] is not None:
                    res[j] = "("+res[j]+'+'+wc2[i]+")"
                else:
                    res[j] = wc2[i]
                i += 1
            yield [i for i in res if i]


IndentationError: expected an indented block after 'if' statement on line 30 (38762751.py, line 40)

In [638]:
abc_symb={'aa':-1, 'bb':-1, 'cc':-1}
def_symb={'dd':1, 'ee':1, 'ff':1}

LeftPreLieShuf(symbs[1],abc_symb)

{'aeaa': 2,
 'aaea': 2,
 'aaae': -4,
 'aebb': 2,
 'abeb': 2,
 'bbae': -2,
 'babe': -2,
 'aecc': 2,
 'acec': 2,
 'ccae': -2,
 'cace': -2,
 'afaa': 2,
 'aafa': 2,
 'aaaf': -4,
 'afbb': 2,
 'abfb': 2,
 'bbaf': -2,
 'babf': -2,
 'afcc': 2,
 'acfc': 2,
 'ccaf': -2,
 'cacf': -2,
 'bdaa': 2,
 'bada': 2,
 'aabd': -2,
 'abad': -2,
 'bdbb': 2,
 'bbdb': 2,
 'bbbd': -4,
 'bdcc': 2,
 'bcdc': 2,
 'ccbd': -2,
 'cbcd': -2,
 'bfaa': 2,
 'bafa': 2,
 'aabf': -2,
 'abaf': -2,
 'bfbb': 2,
 'bbfb': 2,
 'bbbf': -4,
 'bfcc': 2,
 'bcfc': 2,
 'ccbf': -2,
 'cbcf': -2,
 'cdaa': 2,
 'cada': 2,
 'aacd': -2,
 'acad': -2,
 'cdbb': 2,
 'cbdb': 2,
 'bbcd': -2,
 'bcbd': -2,
 'cdcc': 2,
 'ccdc': 2,
 'cccd': -4,
 'ceaa': 2,
 'caea': 2,
 'aace': -2,
 'acae': -2,
 'cebb': 2,
 'cbeb': 2,
 'bbce': -2,
 'bcbe': -2,
 'cecc': 2,
 'ccec': 2,
 'ccce': -4}

In [587]:
abc_symb={'aa':-1, 'bb':-1, 'cc':-1, 'dd':1, 'ee':1, 'ff':1}


In [591]:
mysymb={'aa':1, 'bb':1, 'cc':1, 'dd':-2, 'ee':-2, 'ff':-2}


L12=Lshufsymbs(symbs[1],symbs[2],"ok_phi2")
R12=Rshufsymbs(symbs[1],symbs[2],"ok_phi2")
L21=Lshufsymbs(symbs[2],symbs[1],"ok_phi2")
R21=Rshufsymbs(symbs[2],symbs[1],"ok_phi2")

print([k for k in L12 if k not in symbs[3]])


print([k for k in R12 if k not in symbs[3]])
print([k for k in L21 if k not in symbs[3]])
print([k for k in R21 if k not in symbs[3]])

#print("aeee" in Lshufsymbs(mysymb,symbs[1]))


print("aeee" in Rshufsymbs(symbs[1],mysymb))

#print("aeee" in ))





#print(Rshufsymbs(def_symb,symbs[1]))

['aaaaee', 'aceeee', 'aaeeee', 'aaaaff', 'aaffff', 'abffff', 'bbbbdd', 'bbdddd', 'bcdddd', 'bbbbff', 'baffff', 'bbffff', 'ccccdd', 'cbdddd', 'ccdddd', 'ccccee', 'cceeee', 'caeeee']
['aaaaee', 'cccaee', 'aceeee', 'caeeee', 'aaeeee', 'aaaaff', 'bbbaff', 'aaffff', 'abffff', 'baffff', 'bbbbdd', 'cccbdd', 'bbdddd', 'bcdddd', 'cbdddd', 'aaabff', 'bbbbff', 'bbffff', 'bbbcdd', 'ccccdd', 'ccdddd', 'aaacee', 'ccccee', 'cceeee']
['aaaaee', 'aaaecd', 'aaaece', 'aaacee', 'aaaaff', 'aaafbd', 'aaafbf', 'aaabff', 'bbbbdd', 'bbbdcd', 'bbbcdd', 'bbbdce', 'bbbfae', 'bbbfaf', 'bbbaff', 'bbbbff', 'cccdbd', 'cccbdd', 'cccdbf', 'ccccdd', 'ccceae', 'cccaee', 'ccceaf', 'ccccee', 'bbdddd', 'bcdddd', 'cbdddd', 'ccdddd', 'caeeee', 'cceeee', 'aaeeee', 'aceeee', 'aaffff', 'abffff', 'baffff', 'bbffff']
['aaaaee', 'aaaecd', 'aaaece', 'aaacee', 'aaaaff', 'aaafbd', 'aaafbf', 'aaabff', 'bbbbdd', 'bbbdcd', 'bbbcdd', 'bbbdce', 'bbbfae', 'bbbfaf', 'bbbaff', 'bbbbff', 'cccdbd', 'cccbdd', 'cccdbf', 'ccccdd', 'ccceae', 'cccae

In [538]:
"aeee" in Lshufsymbs(symbs[1],symbs3[1])

False

In [539]:
Rshufsymbs(symbs[1],symbs3[1])

{'aeab': -2,
 'aaeb': -4,
 'aeba': -2,
 'abea': -2,
 'baea': -2,
 'aebc': -2,
 'abec': -2,
 'baec': -2,
 'aecb': -2,
 'aceb': -4,
 'caeb': -4,
 'aeca': -2,
 'acea': 2,
 'caea': 2,
 'aeac': -2,
 'aaec': -4,
 'aeaa': 4,
 'aaea': 8,
 'aebb': 4,
 'abeb': 4,
 'baeb': 4,
 'aecc': 4,
 'acec': 2,
 'caec': 2,
 'afab': -2,
 'aafb': -4,
 'afba': -2,
 'abfa': 2,
 'bafa': 2,
 'afbc': -2,
 'abfc': -4,
 'bafc': -4,
 'afcb': -2,
 'acfb': -2,
 'cafb': -2,
 'afca': -2,
 'acfa': -2,
 'cafa': -2,
 'afac': -2,
 'aafc': -4,
 'afaa': 4,
 'aafa': 8,
 'afbb': 4,
 'abfb': 2,
 'bafb': 2,
 'afcc': 4,
 'acfc': 4,
 'cafc': 4,
 'bdab': -2,
 'badb': -2,
 'abdb': -2,
 'bdba': -2,
 'bbda': -4,
 'bdbc': -2,
 'bbdc': -4,
 'bdcb': -2,
 'bcdb': 2,
 'cbdb': 2,
 'bdca': -2,
 'bcda': -4,
 'cbda': -4,
 'bdac': -2,
 'badc': -2,
 'abdc': -2,
 'bdaa': 4,
 'bada': 4,
 'abda': 4,
 'bdbb': 4,
 'bbdb': 8,
 'bdcc': 4,
 'bcdc': 2,
 'cbdc': 2,
 'bfab': -2,
 'bfba': -2,
 'bbfa': -4,
 'bfbc': -2,
 'bbfc': -4,
 'bfcb': -2,
 'bcfb': -2,
 'c

In [540]:
tinysymb={'aa':1, 'bb':1, 'cc':1, 'dd':1, 'ee':1, 'ff':1}
abc_symb={'aa':1, 'bb':1, 'cc':1}
def_symb={'dd':1, 'ee':1, 'ff':1}

In [541]:
#print(symbs[3])
print(symbs[2])

{'aaae': 16, 'aaaf': 16, 'bbbd': 16, 'bbbf': 16, 'cccd': 16, 'ccce': 16, 'bddd': 8, 'cddd': 8, 'ceee': 8, 'aeee': 8, 'afff': 8, 'bfff': 8}


In [542]:
from pprint import pprint
pprint(Rshufsymbs(symbs[1],symbs[2]))

{'aaaaee': -128,
 'aaaaef': -128,
 'aaaafe': -128,
 'aaaaff': -128,
 'aaabde': -32,
 'aaabdf': -32,
 'aaabfe': -32,
 'aaabff': -32,
 'aaacde': -32,
 'aaacdf': -32,
 'aaacee': -32,
 'aaacef': -32,
 'aaaeae': -96,
 'aaaeaf': -96,
 'aaafae': -96,
 'aaafaf': -96,
 'aabade': -32,
 'aabadf': -32,
 'aabafe': -32,
 'aabaff': -32,
 'aabdae': -32,
 'aabdaf': -32,
 'aabfae': -32,
 'aabfaf': -32,
 'aacade': -32,
 'aacadf': -32,
 'aacaee': -32,
 'aacaef': -32,
 'aacdae': -32,
 'aacdaf': -32,
 'aaceae': -32,
 'aaceaf': -32,
 'aaeaae': -64,
 'aaeaaf': -64,
 'aaeeee': -96,
 'aaeefe': -32,
 'aaefee': -32,
 'aaefff': -32,
 'aafaae': -64,
 'aafaaf': -64,
 'aafeee': -32,
 'aafeff': -32,
 'aaffef': -32,
 'aaffff': -96,
 'abaade': -32,
 'abaadf': -32,
 'abaafe': -32,
 'abaaff': -32,
 'abadae': -32,
 'abadaf': -32,
 'abafae': -32,
 'abafaf': -32,
 'abbbed': -32,
 'abbbef': -32,
 'abbbfd': -32,
 'abbbff': -32,
 'abbebd': -32,
 'abbebf': -32,
 'abbfbd': -32,
 'abbfbf': -32,
 'abdaae': -32,
 'abdaaf': -32,
 'ab

In [280]:
#pprint(symbs[3])

pprint([k for k in symbs[3] if (k.count('d') == 0 and k.count('b') == 0 and k.count('c') == 0 and k.count('e') == 0)])

['aaffaf',
 'aafaff',
 'aafaaf',
 'afafaf',
 'afaaff',
 'afaaaf',
 'aaafff',
 'aaafaf',
 'afffaf',
 'affaff',
 'affaaf',
 'afafff',
 'afffff',
 'aaaaaf']


In [ ]:
['aaffaf',
 'aafaff',
 'aafaaf',
 'afafaf',
 'afaaff',
 'afaaaf',
 'aaafff',
 'aaafaf',
 'afffaf',
 'affaff',
 'affaaf',
 'afafff',
 'afffff',
 'aaaaaf']


000001

[000011]-> aaaaff
[001111] -> aaffff

000101
000111

001001
001011
001101


010001
010011
010101
010111
011001
011011
011101
011111



In [341]:
unsymb={'aa':-4/3, 'ab':2/3, 'ac':2/3, 'ad':0, 'ae':-2, 'af':-2,
        'ba':2/3, 'bb':-4/3, 'bc':2/3, 'bd':-2, 'be':0, 'bf':-2,
        'ca':2/3, 'cb':2/3, 'cc':-4/3, 'cd':-2, 'ce':-2, 'cf':0,
        'da':0, 'db':1, 'dc':1, 'dd':-2/3, 'de':-1, 'df':-1,
        'ea':1, 'eb':0, 'ec':1, 'ed':-1, 'ee':-2/3, 'ef':-1,
        'fa':1, 'fb':1, 'fc':0, 'ef':-1, 'fe':-1, 'ff':-2/3}

In [342]:
import pprint
pprint.pprint(shufsymbs(unsymb,unsymb))

{'aaae': 16.0,
 'aaaf': 16.0,
 'aabd': 5.333333333333333,
 'aabf': 0.0,
 'aacd': 5.333333333333333,
 'aace': 0.0,
 'aaee': 17.77777777777778,
 'aaff': 17.77777777777778,
 'abae': -5.333333333333333,
 'abaf': 0.0,
 'abbd': -5.333333333333333,
 'abbf': 0.0,
 'abcd': -5.333333333333333,
 'abce': -5.333333333333333,
 'abdd': -0.8888888888888888,
 'abff': 15.11111111111111,
 'acae': 0.0,
 'acaf': -5.333333333333333,
 'acbd': -5.333333333333333,
 'acbf': -5.333333333333333,
 'accd': -5.333333333333333,
 'acce': 0.0,
 'acdd': -0.8888888888888888,
 'acee': 15.11111111111111,
 'aeae': 5.777777777777779,
 'aeaf': 6.666666666666665,
 'aecd': 6.666666666666667,
 'aece': 3.1111111111111107,
 'aeee': 8.0,
 'afae': 6.666666666666666,
 'afaf': 5.777777777777779,
 'afbd': 8,
 'afbf': 3.1111111111111107,
 'afff': 8.0,
 'baae': -5.333333333333333,
 'baaf': 0.0,
 'babd': -5.333333333333333,
 'babf': 0.0,
 'bacd': -5.333333333333333,
 'bace': -5.333333333333333,
 'baee': -0.8888888888888888,
 'baff': 15.11

In [ ]:
def LHalfShufPowCoef(symb,num,cond=None):
    if num == 0:
        return Symb({})
    else:
        i=1
        mysymb=symb
        while i < num:   
            mysymb=Lshufsymbs(mysymb,symb)
            i += 1      
    return Symb(mysymb)

def RHalfShufPowCoef(symb,num,cond=None):
    if num == 0:
        return Symb({})
    else:
        i=1
        mysymb=symb
        while i < num:   
            mysymb=Rshufsymbs(mysymb,symb)
            i += 1      
    return Symb(mysymb)

def LHalfShufPow(word,num,cond=None):
    if num == 0:
        return Symb({})
    else:
        i=1
        symb=Symb({word:1})
        mysymb=symb
        while i < num:   
            mysymb=Lshufsymbs(mysymb,symb)
            i += 1      
    return Symb(mysymb)

def RHalfShufPow(word,num,cond=None):
    if num == 0:
        return Symb({})
    else:
        i=1
        symb=Symb({word:1})
        mysymb=symb
        while i < num:   
            mysymb=Rshufsymbs(mysymb,symb)
            i += 1      
    return Symb(mysymb)